In [17]:
import json
import pandas as pd
from a2ginputstream import A2GClient
import random
import time
import datetime
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

## read data

In [24]:
print("reading data...")

print ("reading users...")
users_str = open("../data/raw/mock_users.json", "r").read()
users = json.loads(users_str)

print ("reading machines...")
machines_str = open("../data/raw/mock_machines.json", "r").read()
machines = json.loads(machines_str)


reading data...
reading users...
reading machines...


In [25]:
df_users = pd.DataFrame(users)
df_users.head()

,name,role
0,John Doe,admin
1,Jane Doe,user
2,John Smith,user
3,Smith Alfred,user
4,Richard Alfredo,user


## merge data

In [26]:
import datetime


print("merging data...")

# 70, 25, 25%
def get_status():
    r = random.randint(0, 100)
    if r < 50: return "active"
    elif r < 75: return "inactive"
    else: return "error"

df_users["machines"] = df_users["name"].apply(lambda _: machines[random.randint(0, len(machines)-1)])
df_users["status"] = df_users["name"].apply(lambda _: get_status())
df_users["ton"] = df_users["machines"].apply(lambda _: random.randint(0, 100))
df_users["start_on"] = df_users["machines"].apply(lambda _: (datetime.datetime.utcnow() - datetime.timedelta(minutes=5)).isoformat())
df_users["end_on"] = df_users["machines"].apply(lambda _: datetime.datetime.utcnow().isoformat())
print("done merging data")

merging data...
done merging data


In [27]:
df_users.head()

,name,role,machines,status,ton,start_on,end_on
0,John Doe,admin,C302,active,82,2024-10-28T20:29:04.448645,2024-10-28T20:34:04.449645
1,Jane Doe,user,C301,error,76,2024-10-28T20:29:04.448645,2024-10-28T20:34:04.449645
2,John Smith,user,C303,active,69,2024-10-28T20:29:04.448645,2024-10-28T20:34:04.449645
3,Smith Alfred,user,C301,active,57,2024-10-28T20:29:04.448645,2024-10-28T20:34:04.449645
4,Richard Alfredo,user,C301,active,97,2024-10-28T20:29:04.448645,2024-10-28T20:34:04.449645


## simulate error ETL and long time process

In [28]:
rand_error = random.randint(0, 100)
if rand_error < 30:
    print("error")
    raise Exception("Error")

rand_ltp = random.randint(0, 100)
if rand_ltp < 30:
    for _ in range(0, 500):
        time.sleep(1) 

## insert data

In [29]:
print("sending data...")
USER_PAT = "8f8ff571c7344ce9b6fae09f1ff9c5b1"
IKEY = "1c01a94bea5948faabda"
client = A2GClient(token=USER_PAT)
client.insert_data(IKEY, df_users.to_dict("records"))



2024-10-28 17:34:07,516 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443


sending data...


2024-10-28 17:34:08,051 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/1c01a94bea5948faabda HTTP/1.1" 200 None
2024-10-28 17:34:08,067 - INFO - Getting inputstream 1c01a94bea5948faabda from A2G...
2024-10-28 17:34:08,070 - WARNING - Inputstream discovered
2024-10-28 17:34:08,076 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-28 17:34:08,561 - DEBUG - https://v2streams.a2g.io:443 "POST /Data/Insert HTTP/1.1" 200 None
2024-10-28 17:34:08,569 - INFO - batch 1, docs: [0 - 31] - 200 - {"message":"Data Received Success, DataToy inputstream","data":null,"success":true,"errors":null}
